# Example of the `aitlas` toolbox in the context of multi label image classification

This notebook shows a sample implementation of a multi label image classification using the `aitlas` toolbox using the Planet UAS multi label dataset.

In [ ]:
from aitlas.datasets import PlanetUASMultiLabelDataset
from aitlas.models import ResNet50MultiLabel
from aitlas.transforms import ResizeCenterCropFlipHVToTensor, ConvertToRGBResizeCenterCropToTensor
from aitlas.utils import image_loader

## Load the dataset

In [ ]:
dataset_config = {
    "data_dir": "./data/PlanetUAS/images",
    "csv_file": "./data/PlanetUAS/multilabels.txt"
}
dataset = PlanetUASMultiLabelDataset(dataset_config)

## Show images from the dataset

In [ ]:
fig1 = dataset.show_image(1000)
fig2 = dataset.show_image(30)
fig3 = dataset.show_batch(15)

## Inspect the data

In [ ]:
dataset.show_samples()

In [ ]:
dataset.data_distribution_table()

In [ ]:
fig = dataset.data_distribution_barchart()

## Load train and test splits

In [ ]:
train_dataset_config = {
    "batch_size": 16,
    "shuffle": True,
    "num_workers": 4,
    "data_dir": "./data/PlanetUAS/images",
    "csv_file": "./data/PlanetUAS/train.csv"
}

train_dataset = PlanetUASMultiLabelDataset(train_dataset_config)
train_dataset.transform = ResizeCenterCropFlipHVToTensor()

## Setup and create the model for training

In [ ]:
epochs = 10
model_directory = "./experiments/PlanetUAS"
model_config = {
    "num_classes": 17, 
    "learning_rate": 0.0001,
    "pretrained": True, 
    "threshold": 0.5, 
    "metrics": ["accuracy", "precision", "recall", "f1_score"]
}
model = ResNet50MultiLabel(model_config)
model.prepare()

## Training and evaluation

In [ ]:
model.train(
    train_dataset=train_dataset,
    epochs=epochs,
    model_directory=model_directory,
    run_id='1',
)

## Predictions

In [ ]:
model_path = "./experiments/PlanetUAS/checkpoint.pth.tar"
#labels = PlanetUASMultiLabelDataset.labels
labels = ["haze", "primary", "agriculture", "clear", "water", "habitation", "road", "cultivation", "slash_burn",
          "cloudy", "partly_cloudy", "conventional_mine", "bare_ground", "artisinal_mine", "blooming",
          "selective_logging", "blow_down"]
transform = ConvertToRGBResizeCenterCropToTensor()
model.load_model(model_path)

image = image_loader('./data/predict/image1.tif')
fig = model.predict_image(image, labels, transform)

image = image_loader('./data/predict/image2.tif')
fig = model.predict_image(image, labels, transform)

image = image_loader('./data/predict/image3.tif')
fig = model.predict_image(image, labels, transform)

image = image_loader('./data/predict/image4.tif')
fig = model.predict_image(image, labels, transform)